In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("../input/amazonearphonesreviews/AllProductReviews.csv")
print(df.shape)
df.head()

In [ ]:
y = df["ReviewStar"]

Use the gensim library to prepare data.

In [ ]:
from gensim.parsing.preprocessing import preprocess_string, strip_punctuation, strip_short
from gensim.parsing.preprocessing import strip_multiple_whitespaces, strip_numeric, remove_stopwords
#import spacy

#nlp = spacy.load("en_core_web_sm")

In [ ]:
reviews = df["ReviewBody"].values.tolist()

In [ ]:
CUSTOM_FILTERS = [lambda x: x.lower(), strip_punctuation,
                  strip_numeric]

texts = [preprocess_string(review, CUSTOM_FILTERS) for review in reviews]

Load the glove model.

In [ ]:
import gensim.downloader as api
glove = api.load('glove-wiki-gigaword-300')

For each review, for each token, get the word vector, then compute the average and append to list X.

In [ ]:
import numpy as np

In [ ]:
X = []

for text in texts:
    vecs = []
    for word in text:
        try:
            vecs.append(glove[word])
        except:
            continue
    X.append(np.mean(vecs, axis = 0))

In [ ]:
len(X)

Each average word vector has dimension 300.

In [ ]:
X[0].shape[0]

Some reviews have 0 words after processing. Fill-in these reviews with zeros.

In [ ]:
missing_indices = []

for i in range(len(X)):
    if X[i].shape != (300,):
        missing_indices.append(i)

In [ ]:
print(len(missing_indices))

In [ ]:
for idx in missing_indices:
    X[idx] = np.zeros(shape = (300,))

To boost performance, use predictions from Vader as features.

In [ ]:
import nltk
nltk.download('vader_lexicon')

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [ ]:
sia = SentimentIntensityAnalyzer()
ss = sia.polarity_scores(reviews[0])
print(ss)

In [ ]:
for i in range(len(X)):
    ss = sia.polarity_scores(reviews[i])
    ss = list(ss.values())
    X[i] = np.concatenate((X[i], np.array(ss)))

There are 5 classes to predict.

In [ ]:
y.value_counts()

Make a train-validation split.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_validation, y_train, y_validation = train_test_split(X, y, random_state = 34, stratify = y)

In [ ]:
np.array(X).shape

Use RepeatedStratifiedKFold to estimate model performance on the training set.

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

pipelines = []
pipelines.append(('Logistic Regression', Pipeline([('imp', SimpleImputer()),
                                                   ('sc', StandardScaler()),
                                                   ('LR', SGDClassifier(loss = "log"))] )))
pipelines.append(('Support Vector Machine', Pipeline([('imp', SimpleImputer()), 
                                                      ('sc', StandardScaler()),
                                                      ('SV', SGDClassifier())] )))
pipelines.append(('Naive Bayes', Pipeline([('imp', SimpleImputer()), 
                                           ('sc', StandardScaler()),
                                           ('NB', GaussianNB())] )))
pipelines.append(('Decision Tree', Pipeline([('imp', SimpleImputer()), 
                                             ('sc', StandardScaler()),
                                             ('Tree', DecisionTreeClassifier())] )))

results = []
names = []

for name, model in pipelines:
    kfold = RepeatedStratifiedKFold(n_splits = 3, random_state = 34)
    cvresults = cross_val_score(model, X_train, y_train, cv = kfold, scoring = "accuracy")
    results.append(cvresults)
    names.append(name)
    msg ="%s: %f (%f)" % (name, cvresults.mean(), cvresults.std())
    print(msg)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.boxplot(results, labels = names)
plt.xticks(rotation = 'vertical')
plt.tight_layout()
plt.show()

In [ ]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

pipelines = []
pipelines.append(('Support Vector Machine', Pipeline([('imp', SimpleImputer()),
                                                   ('sc', StandardScaler()),
                                                   ('SVM', SVC())] )))
pipelines.append(('Random Forest', Pipeline([('imp', SimpleImputer()), 
                                                      ('sc', StandardScaler()),
                                                      ('RF', RandomForestClassifier())] )))
pipelines.append(('KNN', Pipeline([('imp', SimpleImputer()), 
                                           ('sc', StandardScaler()),
                                           ('KNN', KNeighborsClassifier())] )))

results = []
names = []

for name, model in pipelines:
    kfold = RepeatedStratifiedKFold(n_splits = 3, random_state = 34)
    cvresults = cross_val_score(model, X_train, y_train, cv = kfold, scoring = "accuracy")
    results.append(cvresults)
    names.append(name)
    msg ="%s: %f (%f)" % (name, cvresults.mean(), cvresults.std())
    print(msg)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.boxplot(results, labels = names)
plt.xticks(rotation = 'vertical')
plt.tight_layout()
plt.show()

Tune the Support Vector Machine model using a randomized search.

In [ ]:
model = Pipeline([('imp', SimpleImputer()),
                                                   ('sc', StandardScaler()),
                                                   ('SVM', SVC())] )
model.get_params()

In [ ]:
from scipy.stats import uniform

params = {'SVM__C' : uniform(1e-3, 2),
          'SVM__gamma' : uniform(1e-6, 1)}

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

random_search = RandomizedSearchCV(model, params, n_iter = 70, cv = 5, random_state = 6)
random_search.fit(X_train, y_train)

print("Best Score: ", random_search.best_score_)
print("Best Parameters: ", random_search.best_params_)

The default hyperparamters were better than those found in a randomized search.

Fit the model to the training data then estimate performance on the validation set.

In [ ]:
model.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_validation)

print(classification_report(y_validation, y_pred))

In [ ]:
from sklearn.metrics import confusion_matrix

print(confusion_matrix(y_validation, y_pred))

Able to predict 1 and 5 star reviews, but struggle with 2, 3, and 4 star reviews. The model failed to predict a single 2 star review.